In [2]:
import pandas as pd
import os
from rutermextract import TermExtractor
import re
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

from pymystem3 import Mystem
m = Mystem()



from numpy import array

Installing mystem to /home/jovyan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


### Таблица с метаданными корпуса

In [4]:
metadata = pd.read_csv('meta_rubrics_final.tsv', sep = '\t')

In [43]:
len(metadata)

30628

In [5]:
metadata.head()

,path,source,url,date,title,subtitle,author,tags,genre,mapped_rubrics,final_rubrics,number_of_rubrics
0,chrdk.ru/articles/sci_10_salt_lakes.txt,https://chrdk.ru/,https://chrdk.ru/sci/10_salt_lakes,16.08.2017,Десять самых известных соленых озер,NaN,Егор Задереев,География_Экология,Статьи,Науки о земле|Науки о земле,Науки о земле,One
1,chrdk.ru/articles/sci_33_fractures.txt,https://chrdk.ru/,https://chrdk.ru/sci/33_fractures,08.08.2017,33 перелома,Скелет мужчины с 33 переломами нашли археологи...,Екатерина Боровикова,Российская наука_Антропология,Статьи,История|Мусор,История,One
2,chrdk.ru/articles/sci_46_chromosomes.txt,https://chrdk.ru/,https://chrdk.ru/sci/46_chromosomes,04.08.2017,46 — норма?,Считаем хромосомы: сколько человеку для счасть...,Полина Лосева,Генетика_Медицина,Статьи,Физиология человека|Физиология человека,Физиология человека,One
3,chrdk.ru/articles/sci_750gev.txt,https://chrdk.ru/,https://chrdk.ru/sci/750gev,23.08.2016,Несбывшиеся надежды на новую физику,Какие «страшные сценарии» физики хотели забыть...,Екатерина Боровикова,Физика_Интервью_Закрытия,Статьи,Мусор|Мусор|Физика,Физика,One
4,chrdk.ru/articles/sci_alien_anatomy.txt,https://chrdk.ru/,https://chrdk.ru/sci/alien_anatomy,22.05.2017,Анатомия каменных пришельцев,Спецпроект «Чердака» и «Красивой науки»,NaN,Геология_Космос,Статьи,Космос|Науки о земле,Космос,Multi


#### Частотный словарь ####

In [7]:
df = pd.read_csv('freqrnc2011.csv', sep='\t')
freq_dict = df[['Lemma','Freq(ipm)']]
sorted_freq = freq_dict.sort_values(by=['Freq(ipm)'])

In [8]:
len(freq_dict)

52138

In [9]:
sorted_freq.head()

,Lemma,Freq(ipm)
29836,перекачать,0.4
32918,помешанный,0.4
15847,касситерит,0.4
15850,кастаньета,0.4
32896,померанец,0.4


In [10]:
sorted_freq.tail()

,Lemma,Freq(ipm)
51957,я,12684.4
22019,на,15867.3
23328,не,18028.0
3710,в,31374.2
13824,и,35801.8


In [11]:
lemmas = list(freq_dict['Lemma'])
values = list(freq_dict['Freq(ipm)'])

dictionary = dict(zip(lemmas, values))

#### Символьные биграммы ####

In [12]:
with open('bigrams.txt') as fl:
    bi = fl.read()
    
with open('trigrams.txt') as fl:
    tri = fl.read()
    
with open('tetragrams.txt') as fl:
    tetra = fl.read()
    

In [13]:
bigrams = bi.split('\n')
trigrams = tri.split('\n')
tetragrams = tetra.split('\n')

#### Аффиксы ####

In [14]:
with open('suf2.txt') as fl:
    s2 = fl.read()

with open('suf3.txt') as fl:
    s3 = fl.read()
    
with open('suf4.txt') as fl:
    s4 = fl.read()

with open('pref2.txt') as fl:
    p2 = fl.read()

with open('pref3.txt') as fl:
    p3 = fl.read()

with open('pref4.txt') as fl:
    p4 = fl.read()

In [15]:
suf2 = s2.split('\n')
suf3 = s3.split('\n')
suf4 = s4.split('\n')
pref2 = p2.split('\n')
pref3 = p3.split('\n')
pref4 = p4.split('\n')

#### Контекстные слова ####

In [17]:
with open('frequent_words.txt') as fl:
    cont = fl.read()

In [18]:
context = cont.split('\n')
print(context)

['изменение', 'анализ', 'количество', 'концентрация', 'образец', 'объем', 'понятие', 'амплитуда', 'действие', 'использование', 'использовать', 'исследование', 'метод', 'под названием', 'называется', 'называют', 'например', 'развитие', 'разновидность', 'рост', 'система', 'содержание', 'содержать', 'стадия', 'структура', 'термин', 'увеличение', 'уменьшение', 'уровень', 'формирование', 'формировать', 'эффект', 'дефект']


#### Кандидаты Rutermextractor ####

In [19]:
def get_term_candidates(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    terms = []
    term_extractor = TermExtractor()

    for term in term_extractor(text):
        terms.append(term.normalized)
    return terms

#### Предобработка #####

In [21]:
def cand_preprocessing(list_of_candidates):
    term_unigrams = []
    term_bigrams = []
    term_trigrams = []
    term_multigrams = []

    lem_unigrams = []
    lem_bigrams = []
    lem_trigrams = []
    lem_multigrams = []

    for term in list_of_candidates:
        if len(term.split()) == 1:
            term_unigrams.append(term.lower())
            lem_unigrams.append(m.lemmatize(term))
        elif len(term.split()) == 2:
            term_bigrams.append(term.lower())
            lem_bigrams.append(m.lemmatize(term))
        elif len(term.split()) == 3:
            term_trigrams.append(term.lower())
            lem_trigrams.append(m.lemmatize(term))
        else:
            term_multigrams.append(term.lower())
            lem_multigrams.append(m.lemmatize(term))

    lem_uni = []

    for u in lem_unigrams:
        lem_uni.append(u[0])

    lem_bi = []
    for b in lem_bigrams:
        lem_bi.append([b[0],b[2]])

    lem_tri = []
    for t in lem_trigrams:
        lem_tri.append([t[0],t[2],t[4]])

    lem_multi = []

    for mul in lem_multigrams:
        mul = [x for x in mul if x != ' ']
        mul = mul[:-1]
        lem_multi.append(mul)
    return lem_uni, lem_bi, lem_tri, lem_multi

#### Отсев кандидатов ####

In [22]:
def freq_scores(list_of_terms):#проверяем частотность
    scores = []
    global lemmas
    global dictionary
    
    for term in list_of_terms:
        if term not in lemmas:
            scores.append(1)
        elif dictionary.get(term) < 40:
            scores.append(1)
        else:
            scores.append(0)
    return array(scores)

In [23]:
def pref_scores(list_of_terms):#проверяем префиксы
    scores1 = []
    global pref2, pref3, pref4
    
    for term in list_of_terms:
        if term[:-4] in pref4 == True:
            scores1.append(1)
        elif term[-3:] in pref3 == True:
            scores1.append(1)
        elif term[:-2] in pref2 == True:
            scores1.append(1)
        else:
            scores1.append(0)
    return array(scores1)

In [24]:
def suf_scores(list_of_terms):#проверяем суффиксы
    scores2 = []
    global suf2
    global suf3
    global suf4
    
    for term in lem_uni:
        t = stemmer.stem(term)
        if t[:4] in suf4 == True:
            scores2.append(1)
        elif t[:3] in suf3 == True:
            scores2.append(1)
        elif t[:2] in suf2 == True:
            scores2.append(1)
        else:
            scores2.append(0)
    return array(scores2)

In [25]:
def get_scores(list_of_terms):
    score1 = freq_scores(list_of_terms)
    score2 = suf_scores(list_of_terms)
    score3 = pref_scores(list_of_terms)
    
    total = score1 + score2 + score3
    cand_scores = dict(zip(list_of_terms, total))
    
    winners = []
    for candidate in cand_scores:
        if cand_scores.get(candidate) != 0:
            winners.append(candidate)
        else:
            continue
    
    return cand_scores, winners

# КОД #

## Извлекаем термины ##

In [26]:
from tqdm import tqdm

In [27]:
results = []
for path in tqdm(metadata['path']):
    try:
        candidates = get_term_candidates('./popular_science_texts_store/'+ path)
        lem_uni, lem_bi, lem_tri, lem_multi = cand_preprocessing(candidates)
        scores, winners = get_scores(lem_uni)
        results.append(winners)
    except:
        results.append('No_file')
        continue

  0%|          | 0/30628 [00:00<?, ?it/s]/opt/conda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
100%|██████████| 30628/30628 [2:06:18<00:00,  4.04it/s]  


In [28]:
results[:70]

[['фото',
  'соленость',
  'моно',
  'фламинго',
  'мышьяк',
  'чаща',
  'хиллиер',
  'соль',
  'космос',
  'каспий',
  'жуан',
  'дон',
  'водоем',
  'бактерия',
  'фотодом',
  'геология',
  'shutterstock',
  'nasa',
  'flickr',
  'ют',
  'эфиопия',
  'цитирование',
  'фотограф',
  'фосфор',
  'фитопланктон',
  'улан-удэ',
  'удобрение',
  'туризм',
  'сырдарья',
  'сторонник',
  'стая',
  'специфика',
  'солончак',
  'сми',
  'религиоведение',
  'рачок',
  'рана',
  'пустыня',
  'простой',
  'популярность',
  'подборка',
  'перестройка',
  'осетр',
  'осадки',
  'обыватель',
  'ноль',
  'насыпь',
  'мор',
  'микроорганизм',
  'метаболизм',
  'местоположение',
  'местность',
  'матмодель',
  'марс',
  'мара',
  'литий',
  'корма',
  'корм',
  'контраст',
  'каротиноид',
  'исследователь',
  'испарение',
  'закат',
  'джон',
  'грунт',
  'волга',
  'водоросль',
  'видеорепортаж',
  'астробиология',
  'артемий',
  'артемия',
  'аргентина',
  'арать',
  'арал',
  'антарктида',
  'амударь

In [208]:
 flat_results = [term for sublist in results for term in sublist]

In [209]:
len(flat_results)

901090

# Постобработка

### Снизили пороговый показатель частотности до 20

In [37]:
further_results = []
counter = []

In [39]:
for terms in tqdm(results):
    for term in terms:
        if term not in lemmas:
            counter.append(term)
        elif dictionary.get(term) < 20:
            counter.append(term)
        else:
            continue
            
    further_results.append(counter)
    counter = []

100%|██████████| 30628/30628 [06:06<00:00, 83.68it/s]


In [63]:
print(further_results[:70])

[['соленость', 'моно', 'фламинго', 'мышьяк', 'чаща', 'хиллиер', 'каспий', 'жуан', 'водоем', 'бактерия', 'фотодом', 'геология', 'shutterstock', 'nasa', 'flickr', 'ют', 'эфиопия', 'цитирование', 'фотограф', 'фосфор', 'фитопланктон', 'улан-удэ', 'удобрение', 'туризм', 'сырдарья', 'стая', 'солончак', 'сми', 'религиоведение', 'рачок', 'простой', 'подборка', 'осетр', 'осадки', 'обыватель', 'ноль', 'насыпь', 'мор', 'микроорганизм', 'метаболизм', 'местоположение', 'матмодель', 'марс', 'мара', 'литий', 'корма', 'корм', 'контраст', 'каротиноид', 'испарение', 'закат', 'джон', 'грунт', 'волга', 'водоросль', 'видеорепортаж', 'астробиология', 'артемий', 'артемия', 'аргентина', 'арать', 'арал', 'антарктида', 'амударья', 'science', 'salexey', 'nandaro', 'kurioziteti123', 'gsfc', 'djpmapleferryman'], ['телега', 'наталия', 'ямник', 'березина', 'скелет', 'позвоночник', 'швейцария', 'чердак', 'погребение', 'перелом', 'дтп', 'германия', 'возница', 'чаща', 'умерший', 'травма', 'ставрополье', 'сородич', 'соо

### Убрали слова с латинскими символами##

In [60]:
no_latin_terms = []
r = re.compile("^[а-яА-Я]+$")

for terms in tqdm(further_results):
    russian = [w for w in filter(r.match, terms) if w != 'No_file']
    no_latin_terms.append(russian)
    

100%|██████████| 30628/30628 [00:00<00:00, 53925.38it/s]


In [64]:
print(no_latin_terms[:100])

[['соленость', 'моно', 'фламинго', 'мышьяк', 'чаща', 'хиллиер', 'каспий', 'жуан', 'водоем', 'бактерия', 'фотодом', 'геология', 'ют', 'эфиопия', 'цитирование', 'фотограф', 'фосфор', 'фитопланктон', 'удобрение', 'туризм', 'сырдарья', 'стая', 'солончак', 'сми', 'религиоведение', 'рачок', 'простой', 'подборка', 'осетр', 'осадки', 'обыватель', 'ноль', 'насыпь', 'мор', 'микроорганизм', 'метаболизм', 'местоположение', 'матмодель', 'марс', 'мара', 'литий', 'корма', 'корм', 'контраст', 'каротиноид', 'испарение', 'закат', 'джон', 'грунт', 'волга', 'водоросль', 'видеорепортаж', 'астробиология', 'артемий', 'артемия', 'аргентина', 'арать', 'арал', 'антарктида', 'амударья'], ['телега', 'наталия', 'ямник', 'березина', 'скелет', 'позвоночник', 'швейцария', 'чердак', 'погребение', 'перелом', 'дтп', 'германия', 'возница', 'чаща', 'умерший', 'травма', 'ставрополье', 'сородич', 'соотечественник', 'скотоводство', 'россия', 'переворот', 'охра', 'останки', 'новшество', 'нии', 'находка', 'майкопец', 'курган',

### Убираем имена собственные и географические названия

In [69]:
raw_geo = pd.read_csv('geo_names.tsv', sep = '\t')

In [67]:
with open('ner_corpora_extracted.txt') as fl:
    raw_ner = fl.read()

In [80]:
geo = list(raw_geo['location'])

In [81]:
geo = [term.split(' ') for term in geo if term.split(' ')]

In [82]:
geo = [term.lower() for sublist in geo for term in sublist]

In [89]:
ner = raw_ner.lower().split('\n')

In [91]:
ner = [name.split(' ') for name in ner if name.split(' ')]

In [92]:
ner = [name for sublist in ner for name in sublist]

In [94]:
stopwords = list(set(geo + ner))

In [96]:
len(stopwords)

18304

In [97]:
no_stop_words = []

for terms in no_latin_terms:
    terms = [term for term in terms if term not in stopwords]
    no_stop_words.append(terms)

In [138]:
print(no_stop_words[:100])

[['соленость', 'моно', 'фламинго', 'мышьяк', 'чаща', 'хиллиер', 'каспий', 'жуан', 'водоем', 'бактерия', 'геология', 'ют', 'цитирование', 'фотограф', 'фосфор', 'фитопланктон', 'удобрение', 'туризм', 'сырдарья', 'стая', 'солончак', 'сми', 'религиоведение', 'рачок', 'простой', 'подборка', 'осетр', 'осадки', 'обыватель', 'ноль', 'насыпь', 'мор', 'микроорганизм', 'метаболизм', 'местоположение', 'матмодель', 'марс', 'мара', 'литий', 'корма', 'корм', 'контраст', 'каротиноид', 'испарение', 'закат', 'джон', 'грунт', 'волга', 'водоросль', 'видеорепортаж', 'астробиология', 'арать', 'амударья'], ['телега', 'наталия', 'ямник', 'скелет', 'позвоночник', 'чердак', 'погребение', 'перелом', 'дтп', 'возница', 'чаща', 'травма', 'ставрополье', 'сородич', 'соотечественник', 'скотоводство', 'переворот', 'охра', 'останки', 'новшество', 'нии', 'находка', 'майкопец', 'курган', 'конечность', 'ключица', 'катакомбника', 'кавказ', 'ископаемое', 'дманиси', 'грудина', 'геном', 'быков', 'артроз', 'артрит', 'антрополог

## Список частотности

In [105]:
flat_list = [term for sublist in no_stop_words for term in sublist]

In [108]:
print(len(flat_list), 'vs', len(set(flat_list)))

524501 vs 71691


In [1]:
unique_terms = set(flat_list)

NameError: name 'flat_list' is not defined

In [112]:
counter = []

In [113]:
for term in tqdm(unique_terms):
    counter.append(flat_list.count(term))


  2%|▏         | 1457/71691 [00:35<28:28, 41.10it/s]


  4%|▍         | 2905/71691 [01:10<27:55, 41.05it/s]


  6%|▌         | 4360/71691 [01:45<27:07, 41.37it/s]


  8%|▊         | 5807/71691 [02:20<26:32, 41.36it/s]


 10%|█         | 7248/71691 [02:55<26:00, 41.29it/s]


...

 93%|█████████▎| 66788/71691 [27:03<01:59, 41.15it/s]


 95%|█████████▌| 68203/71691 [27:38<01:24, 41.13it/s]


 97%|█████████▋| 69615/71691 [28:12<00:50, 41.12it/s]


 99%|█████████▉| 71025/71691 [28:47<00:16, 41.12it/s]


100%|██████████| 71691/71691 [29:03<00:00, 41.12it/s]

In [127]:
freq_terms = list(zip(unique_terms, counter))

In [266]:
freq_dict = dict(zip(unique_terms, counter))

In [128]:
freq_terms.sort(key = lambda freq_terms: freq_terms[1], reverse=True)

In [156]:
print(freq_terms[:2000])

[('чаща', 2023), ('люба', 1935), ('новое', 1886), ('разработчик', 1612), ('гипотеза', 1435), ('видео', 1400), ('орбита', 1387), ('простой', 1295), ('москва', 1286), ('александр', 1285), ('астроном', 1231), ('находка', 1163), ('живой', 1153), ('археолог', 1078), ('робот', 1057), ('малый', 1008), ('соцсеть', 1001), ('марс', 986), ('владимир', 977), ('запуск', 960), ('деньга', 947), ('бактерия', 892), ('днк', 884), ('биолог', 863), ('электрон', 826), ('ком', 818), ('ноль', 799), ('поливать', 736), ('честь', 707), ('толщина', 675), ('телескоп', 674), ('обнаружение', 671), ('излучение', 664), ('чердак', 660), ('счастие', 623), ('джон', 615), ('иллюстрация', 614), ('прототип', 613), ('галактика', 610), ('насекомое', 608), ('ссср', 605), ('смартфон', 603), ('старший', 602), ('нейрон', 573), ('т', 572), ('останки', 571), ('м', 568), ('мкс', 562), ('биология', 549), ('андрей', 539), ('алексей', 530), ('раскопка', 527), ('русаков', 525), ('см', 521), ('мутация', 514), ('водород', 508), ('фотон',

### Доплняем список стоп-слов

In [194]:
new_stop_words2 = ["озеры", "дискомфорт", "корн", "отступление", "титул", "укрытие", "валентин", "пароль", "алтай", "примечание",
                  "приготовление", "непонимание", "обилие", "календарь", "отключение", "маяк", "графство", "подбор", "голландец",
                  "филадельфия", "фила", "недовольство", "сброс", "корректировка", "завоевание", "предназначение", "полдень", "громкость",
                  "а", "кура", "взлом", "консультант", "православие", "юнона", "мощь", "земель", "еписком", "ослабление", "отработка",
                  "зав", "энцелаживать", "обоснование", "арсенал", "странность", "василиевич", "мерка", "вложение", "минобрнаука", "практик",
                  "хроника", "подлодка", "приверженец", "улитка", "меньшинство", "франк", "паутика", "рождество", "подзарядка", "разведение",
                  "виктория", "правота", "заповедник", "показ", "горожанин", "анатолиевич", "ангар", "турок", "прослушивание", "интернетома",
                  "инициатор", "разочарование", "этика", "отвращение", "резюме", "запчасть", "усовершенствование", "рецензия", "аптека", "аэс",
                  "пересмотр", "голубь", "возражение", "пригород", "марсианин", "копенгаген", "спасатель", "литр", "славянин", "снабжение", 
                  "затруднение", "фано", "адресат", "барабан", "разбор", "екатеринбург", "коняев", "средиземноморье", "превосходство", "шедевр",
                  "чума", "табак", "поляк", "кончик", "вдохновение", "икра", "лесна", "многообразие", "камчатка", "репортаж", "перехват", 
                  "отсек", "обобщение", "триумф", "успешность", "мостик", "денис", "мюнхен", "пак", "посылка", "видеоигра", "концерн", "подписка",
                  "христос", "проблематика", "дракон", "хитрость", "маневрирование", "землянин", "светов", "парковка", "извинение", "оптимист",
                  "денге", "преувеличение", "купец", "лайнер", "снятие", "бодроствование", "викторович", "гусев", "роспись", "мультфильм",
                   "митинг", "преследование", "термит", "кинематограф", "шоколад", "утка", "сеанс", "богиня", "запутанность", "меню", "нигерия",
                  "лапка", "детище", "мигрант", "балканы", "оон", "дубна", "тег", "юриевич", "перебор", "кавычка", "активист", "полметра", "мак",
                  "документакция", "иудей", "нетерпение", "заход", "тонкость", "бегство", "воск", "дозаправка", "кинотеатр", "стройка", "копирование",
                   "злоупотребление", "попугай", "заселение", "ганимед", "пояснение", "релиз", "перекресток", "приток", "запястье", "мировоззрение",
                   "кружок", "отслеживание", "пирог", "равенство", "курильщик", "утрата", "психиатр", "продовольствие", "свалка", "безумие",
                   "подборка", "закупка", "отставание", "новизна", "преемник", "сова", "повествование", "растительность", "пиар", "иисус", "разгадка",
                   "грамота", "спойлер", "переворот", "форд", "атлантика", "содействие", "вставка", "тележка", "глазго", "убежище", "сокровище", "курган",
                   "задумка", "бренд", "маятник", "меуары", "географ", "эссе", "счетчик", "скачивание", "профсоюз", "принстон", "тренажер", "томск",
                   "разногласие", "трон", "хлопок", "дирижабль", "вика", "трамп", "аудиозапись", "корзина", "браузер", "спонсор", "езда", "туловище",
                   "каньон", "пиро", "женева", "джонатан", "гм", "фиан", "татарин", "кюри", "аналитика", "картофель", "прицел",
                   "разворот", "раскрытие", "ощупь", "нло", "собиратель", "помидор", "банан", "корона", "дьявол", "ясность", "антарктика",
                   "чечня", "горилла", "роскошь", "соперничество", "ранение", "коралл", "сажа", "стопа", "рычаг", "джунгли", "гуманитарий",
                   "день", "различение", "соотечественник", "авторство", "резина", "розетка", "фигурка", "слуга", "дворянин", "лорд", "баллон",
                   "замер", "мадрид", "безработица", "осмысление", "разброс", "основоположник", "цюрих", "патрулирование", "пробуждение",
                   "подлинность", "совместительство", "пожелание", "лень", "противопоставление", "слепота", "егор", "глеб", "барэа", "фантаст", "зара",
                   "фитнес", "перев", "восхищение", "неполадка", "подмосковье", "сенатор", "обход", "проживание", "коллекционер", "дубай", "получатель",
                   "тошнота", "си", "ик", "киото", "верблюд", "аплодисменты", "виновник", "педаль", "супермаркет", "ожог", "синь", "мошенничество",
                   "преобладание", "коммерсант", "ильич", "заготовка", "изгнание", "уведомление", "тигр", "самостоятельность", "украинец", "похолодание",
                   "висок", "гребень", "небоскреб", "излечение", "нуждающийся", "получас", "галлия", "ранхгис", "хирургия", "сертификат", "мошенник",
                   "самочувствие", "галина", "смелость", "смешение", "кореец", "вак", "запятая", "видеоматериал", "атланта", "зинина", "заправка", "бонус",
                   "первоисточник", "единомышленник", "племянник", "борец", "проволока", "избавление", "кальмар", "приманка", "завещание", "шанхай",
                   "трюк", "гроза", "неэффективность", "барселона", "комментатор", "классик", "верхушка", "клещ", "орангутан", "повтор", "шест",
                   "первенство", "резка", "цк", "холл", "скульптор", "нпо", "седоля", "спячка", "росс", "гидра", "раскол", "пожертвование",
                   "пропитание", "плавник", "стуккей", "пк", "рок", "велосипедист", "сидение", "хан", "бессоница", "павильон", "владимир", 'наталия']

In [195]:
new_stop_words = ['люба', 'новое', 'видео', 'простой', 'чаща', 'малый', 'москва', 'александр', 'находка', 'живой', 'деньга', 'ком','ноль'
                 'ноль', 'чердак', 'счастие', 'старший', 'андрей', 'алексей', 'честь', "джон", "русаков", "фото", "ссср", "костя", "белый",
                  "дмитрий", "михаил", "крыса", "париж", "желающий", "евгений", "екатерина", "основатель", "ближний", "одноклассник", "доставка",
                  "старое", "ося", "иван", "григорий", "крыть", "поль", "побережье", "чужой", "соавтор", "константин", "виктор", "ролик", "дронов",
                  "просмотр", "англ", "видеозапись", "ивтушок", "разнообразие", "василий", "онлайн", "прорыв", "нагрев", "мертвый", "лектор",
                  "правый", "полвека", "шарик", "анатолий", "мария", "обладатель", "мфть", "анна", "горячее", "муравей", "виталий", "марк", "золотой",
                  "рис", "валерий", "файл", "опасение", "берлин", "рубрика", "боева", "леонид", "выпускник", "пчела", "чикаго", "должное", "елизавета",
                  "елена", "кит", "кассинь", "елена", "также", "римляна", "окружающие", "сордержимое", "минус", "одиночка", "сюрприз", "генри", "орел",
                  "долина", "лягушка", "иванович", "илья", "георгий", "максим", "тасс", "паук", "яд", "киев", "тупик", "отрывок", "видеоролиу", "заключать",
                  "арктика", "сериал", "шлем", "овощ", "твиттер", "макака", "еникеев", "ящерица", "британец", "ферма", "стих", "возврат", "прозвище", "боев",
                  "пионер", "курение", "игла", "колледж", "жук", "собрат", "табличка", "теплов", "черепаха", "антон", "закат", "песнь", "новинка", "новосибирск",
                  "овца", "египтянин", "нии", "индеец", "кавказ", "мисис", "ниша", "мини", "сопоставление", "путешественник", "бостон", "сретенка",
                  "видеокамера", "каталог", "пекин", "кирилл", "крым", "нобелевка", "розетт", "скидка", "дельфин", "лука", "энтузиазм", "вячеслав",
                  "голуба", "акула", "олень", "пузырек", "отрыв", "рыбка", "владимирович", "шоу", "последователь", "плавание", "упор", "шерсть", "олень",
                  "зик", "экскурсия", "гигант", "нехватка", "упаковка", "курсы", "кролик", "беспокойство", "выигрыш", "проводы", "неправда", "торонто", "комар",
                  "ирина", "обман", "пентагон", "мед", "даниил", "поломка", "неспособность", "повторение", "звучение", "компромисс", "браслет", "полицейский",
                  "преподавание", "вадим", "пушкин", "магия", "адам", "развлечение", "иерусалим", "медиа", "боб", "наработка", "специализация", "стокгольм",
                  "фотограф", "прикосновение", "соседство", "вторник", "церн", "известность", "птенец", "скептик", "форт", "зоопарк", "галлия", "трос",
                  "догадка", "крокодил", "бег", "сено", "чертеж", "новичок", "одобрение", "аудио", "дарья", "пешеход", "статуя", "ходьба", "тома",
                  "викинг", "библия", "путаница", "казань", "лебедь", "ввс", "уникальность", "распространенность", "отзыв", "совершенство", "современность",
                  "сенсация", "шторм", "мамонт", "клюв", "крушение", "колонка", "скандинавия", "открытость", "предисловие", "редактирование", "правильность",
                  "коррекция", "размах", "угловой", "классика", "влага", "восстание", "неточность", "кукуруза", "несчастие", "механик", "богач", "подделка",
                  "стереотип", "издатель", "орех", "скульптура", "изобилие", "залив", "тау", "д", "довод", "запоминание", "итальянец", "афины",
                  "ю", "анонс", "загрузка", "ярка", "наводнение", "вредитель", "ровер", "копье", "петров", "широта", "кубик", "анкета", "шахматы",
                  "подсказка", "ассистент", "предзаказ", "близнец", "материк", "стоянка", "всплеск", "испанец", "бобров", "жюри", "фотоаппарат", "узор",
                  "стр", "даба", "дебаты", "ураган", "рф", "печь", "паника", "выключение", "исследовательница", "рекордсмен", "мусульманин", "линейка",
                  "урал", "ислам", "торговец", "цунами", "весы", "договоренность", "губка", "пепел", "кон", "перчатка", "дтп", "полоска", "швед", "балл",
                  "соавторство", "большуй", "лабиринт", "соблазн", "мадагаскар", "скобка", "выяснение", "заговор", "предание", "интуиция", "гусеница",
                  "араб", "геннадий", "крах", "рабство", "равнина", "русло", "очаг", "наполеон", "работодатель", "бел", "аукцион", "люк", "боков",
                  "стартапа", "шок", "тревожность", "пересчет", "обложка", "ес", "рой", "аллергия", "маркетинг", "перелом", "превышение", "сидней",
                  "медуза", "недоверие", "волга", "прошествие", "пациентка", "пшеница", "экономия", "сыр", "упадок", "батарейка", "звездочка", "опросник",
                  "нора", "опровержение", "ф", "стадо", "владислав", "клад", "противовес", "пыльца", "партнерство", "триллион", "краситель", "иоанн", 
                  "уха", "сверстник", "иосиф", "изгиб", "освещенность", "хакер", "чаша", "оговорка", "прививка", "пламень", "янтарь", "видимость", "бедность",
                  "данио", "подражание", "архипелаг", "нежелание", "обыватель", "согласование", "коза", "присоединение", "борисович", "докладчик",
                  "спбга", "привлекательность", "наглядность", "приставка", "отравление", "скачок", "закрытие", "бетон", "ухудщение", "срез", "расписание",
                  "средневековье", "развертывание", "львов", "владиовсток", "амстердам"]                  

In [196]:
more_stop_words = new_stop_words + new_stop_words2

In [197]:
len(more_stop_words)

845

### Еще раз отсеиваем кандидатов из дополненного списка стоп-слов

In [198]:
more_clear = []

In [199]:
for terms in tqdm(no_stop_words):
    terms = [term for term in terms if term not in more_stop_words]
    more_clear.append(terms)


100%|██████████| 30628/30628 [00:05<00:00, 5694.64it/s]

In [212]:
flat_more_clear = [term for sublist in more_clear for term in sublist]

In [260]:
type(more_clear[1])

list

### Создаем ранжированный список частотности для полученных терминов, сохраняем полученные данные

In [215]:
unique_more_clear = list(set(flat_more_clear))

In [217]:
u_counter = []

for term in tqdm(unique_more_clear):
    counter.append(flat_more_clear.count(term))


  2%|▏         | 1686/70866 [00:33<22:37, 50.95it/s]


  5%|▍         | 3437/70866 [01:06<21:37, 51.96it/s]


  7%|▋         | 5189/70866 [01:39<20:57, 52.25it/s]


 10%|▉         | 6941/70866 [02:12<20:19, 52.44it/s]


 12%|█▏        | 8691/70866 [02:45<19:43, 52.52it/s]


...

 92%|█████████▏| 65340/70866 [20:35<01:44, 52.90it/s]


 95%|█████████▍| 67056/70866 [21:07<01:12, 52.91it/s]


 97%|█████████▋| 68772/70866 [21:39<00:39, 52.91it/s]


 99%|█████████▉| 70488/70866 [22:12<00:07, 52.91it/s]


100%|██████████| 70866/70866 [22:19<00:00, 52.92it/s]


In [227]:
freq_more_clear = list(zip(unique_more_clear, counter))

In [228]:
freq_more_clear.sort(key = lambda freq_more_clear: freq_more_clear[1], reverse=True)

In [270]:
total_freq = pd.DataFrame(freq_more_clear, columns = ['Term', 'Freq'])

In [274]:
total_freq.to_csv('total_freq.tsv', sep = '\t')

In [200]:
term_dict = {'Terms': more_clear, 'Genre': list(metadata['genre']), 'Rubric': list(metadata['final_rubrics'])}

In [201]:
term_df = pd.DataFrame(term_dict)

In [261]:
term_df.head()

,Genre,Rubric,Terms
0,Статьи,Науки о земле,"[соленость, моно, фламинго, мышьяк, хиллиер, к..."
1,Статьи,История,"[телега, ямник, скелет, позвоночник, погребени..."
2,Статьи,Физиология человека,"[хромосома, деление, анеуплоидия, яйцеклетка, ..."
3,Статьи,Физика,"[достоверность, сигма, коллайдер, убежденность..."
4,Статьи,Космос,"[хондрит, пришелец, ахондрит]"


In [203]:
term_df.to_csv('terms_rubrics_nostopwords.csv', sep = '\t')